In [1]:
! pip install pandas-gbq
! pip install pyarrow 

In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
import pandas as pd
import pyarrow

# global variables
bucket = "eecs6893-stackoverflow-title-body-bucket"    # TODO : replace with your own bucket name
output_directory_wordcount = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/wordcount'.format(bucket)

# output table and columns name
project_id = 'big-data-6893-363219'
output_dataset = 'stackoverflow_title_body_output'                     #the name of your dataset in BigQuery
output_table_wordcount = 'title_body'
columns_name_wordcount = ['title', 'body']

# parameter
IP = 'localhost'    # ip port
PORT = 9001         # port

STREAMTIME = 600     # time that the streaming process runs

# Helper functions
def saveToStorage(rdd, output_directory, columns_name, mode):
    """
    Save each RDD in this DStream to google storage
    Args:
    rdd: input rdd
    output_directory: output directory in google storage
    columns_name: columns name of dataframe
    mode: mode = "overwirte", overwirte the file
    mode = "append", append data to the end of file
    """
    if not rdd.isEmpty():
        (rdd.toDF( columns_name ) \
        .write.save(output_directory, format="json", mode=mode))
        
def saveToBigQuery(sc, output_dataset, output_table, directory):
    """
    Put temp streaming json files in google storage to google BigQuery
    and clean the output files in google storage
    """
    files = directory + '/part-*'
    subprocess.check_call(
        'bq load --source_format NEWLINE_DELIMITED_JSON '
        '--replace '
        '--autodetect '
        '{dataset}.{table} {files}'.format(
            dataset=output_dataset, table=output_table, files=files
        ).split())
    output_path = sc._jvm.org.apache.hadoop.fs.Path(directory)
    output_path.getFileSystem(sc._jsc.hadoopConfiguration()).delete(output_path, True)

def wordCount(words):
    """
    Calculte the count of 5 sepcial words for every 60 seconds (window no overlap)
    You can choose your own words.
    Your should:
    1. filter the words
    2. count the word during a special window size
    3. add a time related mark to the output of each window, ex: a datetime type
    Hints:
        You can take a look at reduceByKeyAndWindow transformation
        Dstream is a serioes of rdd, each RDD in a DStream contains data from a certain interval
        You may want to take a look of transform transformation of DStream when trying to add a time
    Args:
        dstream(DStream): stream of real time tweets
    Returns:
        DStream Object with inner structure (word, (count, time))
    """

    # TODO: insert your code here
    
    # i don't need this 
    #words = words.map(lambda word: word.lower()).filter(lambda word: word not in ["", ''])
    mapped_words = words.map(lambda word: (word, 1))
    reduced_words = mapped_words.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 10, 10)    
    total_word_count = reduced_words.transform(lambda time, wC: wC.map(lambda x: (x[0], x[1],
                                                                                   time.strftime("%m/%d %H:%M:%S"))))
    
    return total_word_count

def savingTry(words, project_id, output_dataset, output_table_wordcount, columns_name_wordcount):
    
#     words = words.transform(lambda time, x: (x[0], x[1], time.strftime("%m/%d %H:%M:%S")))
    
    data = words.collect()
    
    # 1st scanerio to consider: what if there are multiple questions 
    data[1 : ] = [''.join(data[1 : ])]
    
    list_of_list = []
    list_of_list.append(data)
    
    #print(data)
    dfData = pd.DataFrame(list_of_list, columns=columns_name_wordcount)
    print(dfData)
    sTablePath = "{}.{}".format(output_dataset, output_table_wordcount)
    dfData.to_gbq(sTablePath, project_id, if_exists='append')
    
    
if __name__ == '__main__':
    # Spark settings
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("StackOverflowStreamApp")

    # create spark context with the above configuration
    # sc = SparkContext(conf=conf)
    sc = SparkContext.getOrCreate(conf=conf)
    sc.setLogLevel("ERROR")

    # create sql context, used for saving rdd
    sql_context = SQLContext(sc)

    # create the Streaming Context from the above spark context with batch interval size 5 seconds
    ssc = StreamingContext(sc, 10)
    
    # setting a checkpoint to allow RDD recovery
    ssc.checkpoint("~/checkpoint_stackOverflowApp")

    # read data from port 9001
    dataStream = ssc.socketTextStream(IP, PORT)
#     dataStream.pprint()
    
    #print(type(dataStream))
    
    
    words = dataStream.flatMap(lambda line: line.split("|||||||"))
#     wordCount = wordCount(words)
    words.pprint() # this is working

#     wordCount.foreachRDD(lambda rdd: saveToStorage(rdd, output_directory_wordcount, columns_name_wordcount, mode="append"))
    
    # put the temp result in google storage to google BigQuery
    words.foreachRDD(lambda rdd: savingTry(rdd, project_id, output_dataset, output_table_wordcount, columns_name_wordcount))
    
    # saveToBigQuery(sc, output_dataset, output_table_wordcount, output_directory_wordcount)
    
    # start streaming process, wait for 600s and then stop.
    ssc.start()
    time.sleep(STREAMTIME)
    ssc.stop(stopSparkContext=False, stopGraceFully=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/06 05:31:48 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/12/06 05:31:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/12/06 05:31:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/12/06 05:31:48 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
22/12/06 05:32:00 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error running job streaming job 1670304720000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 568, in _list_to_arrays
    columns = _validate_or_indexify_columns(content, columns)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 692, in _vali

                                               title  \
0  Convert a list&lt;Strings&gt; into a list&lt;M...   

                                                body  
0   <p>Consider this:</p><pre><code>List&lt;Strin...  


100%|██████████| 1/1 [00:00<00:00, 1911.72it/s]                     (0 + 1) / 1]
22/12/06 05:32:37 WARN org.apache.spark.streaming.receiver.ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Socket data stream had no more data
22/12/06 05:32:37 ERROR org.apache.spark.streaming.scheduler.ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
22/12/06 05:32:37 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/12/06 05:32:37 WARN org.apache.spark.storage.BlockManager: Block input-0-1670304757400 replicated to only 0 peer(s) instead of 1 peers


                                               title  \
0  How to convert value(Color string) return by f...   

                                                body  
0   <p>I am having a flutter application and web ...  


100%|██████████| 1/1 [00:00<00:00, 8160.12it/s]                     (0 + 1) / 1]
22/12/06 05:32:50 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error running job streaming job 1670304770000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 568, in _list_to_arrays
    columns = _validate_or_indexify_columns(content, columns)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 692, in _validate_or_indexify_columns
    raise AssertionError(
AssertionError: 2 columns passed, passed data had 1 columns

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/lib/spark/python/pyspark/streaming/dstream.py", line

...

                          title  \
0  Translator API for react js    

                                                body  
0   <p>I want to use a translator like a google t...  


100%|██████████| 1/1 [00:00<00:00, 9686.61it/s]                     (0 + 1) / 1]
22/12/06 05:34:30 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error running job streaming job 1670304870000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 568, in _list_to_arrays
    columns = _validate_or_indexify_columns(content, columns)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 692, in _validate_or_indexify_columns
    raise AssertionError(
AssertionError: 2 columns passed, passed data had 1 columns

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/lib/spark/python/pyspark/streaming/dstream.py", line

                                               title  \
0  How to create a list such that each element of...   

                                                body  
0   <p>I have an example dataframe s1</p><pre><co...  


100%|██████████| 1/1 [00:00<00:00, 11554.56it/s]                    (0 + 1) / 1]
22/12/06 05:34:50 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error running job streaming job 1670304890000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 568, in _list_to_arrays
    columns = _validate_or_indexify_columns(content, columns)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 692, in _validate_or_indexify_columns
    raise AssertionError(
AssertionError: 2 columns passed, passed data had 1 columns

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/lib/spark/python/pyspark/streaming/dstream.py", line

KeyboardInterrupt: 